In [ ]:
!pip install imgaug -U

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import imgaug.augmenters as iaa
import imgaug as ia

In [ ]:
tf.random.set_seed(42)
ia.seed(42)

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 128
EPOCHS = 1
IMAGE_SIZE = 72

In [ ]:
rand_aug = iaa.RandAugment(n=3, m=7)

In [ ]:
def augment(images):
  images = tf.cast(images, np.uint8)
  return rand_aug(images=images.numpy())